In [ ]:
!pip install tensorflow
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Define a function to create the model architecture
def create_model():
    model = tf.keras.Sequential()
    model.add(embed)
    return model

In [ ]:
# Define a function to save the model to disk
def save_model(model, filepath):
    model.save(filepath)

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import numpy as np

In [ ]:
# Connect to firebase
cred = credentials.Certificate("/content/serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
# Load data
users = db.collection('users').get()

user_data = []
for doc in users:
    data = doc.to_dict()
    user_data.append(data)

profile_data = []
for doc in users:
    data = doc.to_dict()
    profile_data.append(data['profile'])

In [57]:
# Process data so it can be use

user_data = pd.DataFrame(user_data, columns=['uid'])
profile_data = pd.DataFrame(profile_data, columns=['displayName','skills', 'interests'])
merge_data = pd.merge(user_data, profile_data, left_index=True, right_index=True)

result_data = merge_data[['uid', 'displayName', 'skills', 'interests']]
result_data['skills'] = result_data['skills'].apply(lambda skill_list: ', '.join([skill_dict['name'] for skill_dict in skill_list if skill_dict and 'uid' in skill_dict]) if isinstance(skill_list, list) else 'No Skill')
result_data['interests'] = result_data['interests'].apply(lambda interest_list: ', '.join([interest_dict['name'] for interest_dict in interest_list if interest_dict and 'uid' in interest_dict]) if isinstance(interest_list, list) else 'No Interest')

user_data = pd.DataFrame(result_data)
user_data

,uid,displayName,skills,interests
0,06yJpLuZ79Dbzyky0TQL,Daren Lardez,"Go, Usability Testing, IBM Cloud, React Native...",DevOps Engineer
1,0CZfmWDndS6xjVv6o8j3,Carter Piolli,"AWS, Ensemble Methods, React Native, IBM Cloud",Web Analyst
2,0M4QzQ4i5v3jFm0atItg,Alf Chrestien,"TypeScript, Hive, Clustering, vulnerability as...",Big Data Specialist
3,0Ot2WxvdSFpsKEI7ZJGx,Jakie Kleiser,"Java, Naive Bayes, Objective-C, Oracle Cloud",Enterprise System Integration Specialist
4,0RSTq7aC8ygoQxFrBN3A,Torry Mullane,"React, Kafka, Decision Trees, cloud security",API Developer Specialist
...,...,...,...,...
498,ygDFgGFFUk9Ay64ZNNBH,Beitris Forsdicke,"jQuery, Pig, Random Forest, network security",Web Designer
499,ykht9Jt6mCEk1TSpszKX,Yank Dorant,"Scala, Visual Design, DigitalOcean, iOS Develo...",Android Technical developer
500,z7CKDLeIahP4PwgZXYEm,Clarita Liversley,"Kotlin, Prototyping, Red Hat Cloud, Flutter De...",IT Capacity Planning Specialist
501,zGY1ZQzBMWhiPgXwBHTt,Charmain McMurraya,"JavaScript, Storm, Decision Trees, identity an...",Flutter Developer


In [58]:
# Define a function to generate user stories
def generate_user_stories(user_data):
    user_story = []
    for index, row in user_data.iterrows():
        user_story.append({
            "uid": row['uid'],
            "story": f"I have Skill {row['skills']}, and I'm Interested in {row['interests']}"
        })
    return user_story

In [59]:
# Define a function to find the top N most similar users to a given user
def find_top_similar_users(current_user_uid, user_story, embed, n=10):
    # Check if current user not found
    if user_data.loc[user_data['uid'] == current_user_uid].empty:
        return "Current user not found!"

    # Get the current user's data and story
    current_user = user_data.loc[user_data['uid'] == current_user_uid]
    current_user_story = f"I have Skill {current_user['skills'][0]} , and I'm Interested in {current_user['interests'][0]}"

    # Encode the current user story into a vector
    current_user_vector = embed([current_user_story])

    # Encode all other user stories into vectors and store them in a matrix along with the user uid
    other_user_vectors = []
    other_user_uid = []
    for user in user_story:
        vector = embed([user["story"]])
        other_user_vectors.append(vector)
        other_user_uid.append(user["uid"])
    other_user_matrix = np.array(other_user_vectors)

    # Calculate the similarity scores between the current user vector and all other user vectors in the matrix
    similarity_scores = tf.matmul(other_user_matrix, tf.transpose(current_user_vector))

    # Get the top N most similar users and their scores
    most_similar_users = np.argsort(similarity_scores.numpy().reshape(-1))[::-1][:n]
    most_similar_user_uid = [other_user_uid[i] for i in most_similar_users]
    most_similar_user_scores = similarity_scores.numpy().reshape(-1)[most_similar_users]

    for i in range(1, n):
        print(f"User ID: {most_similar_user_uid[i]}, Similarity Score: {most_similar_user_scores[i]}")

In [60]:
# Generate user stories, create the model, and call it on a dummy input
user_story = generate_user_stories(user_data)
model = create_model()
model(tf.constant([f"I have Skill Google Cloud, Python, Backend, Typescript, and I'm Interested in Backend Developer"]))

# Save the model
save_model(model, "save_model.h5")

In [61]:
# Example usage of the find_top_similar_users function
find_top_similar_users('06yJpLuZ79Dbzyky0TQL', user_story, embed, n=len(user_story))

User ID: DlmikRZzUud87ZVHWPBv, Similarity Score: 0.8476112484931946
User ID: GYWVAv9MsocnhgVqZHfn, Similarity Score: 0.7643425464630127
User ID: yNY9IhalltJZ02tQU0ix, Similarity Score: 0.7564221024513245
User ID: J5GkwKqxI217nMLNdY1B, Similarity Score: 0.755081057548523
User ID: OtatlU0hDcex4Cn5dRKN, Similarity Score: 0.7529898285865784
User ID: vbxf5J04EfvCznIapW3H, Similarity Score: 0.7501305937767029
User ID: AvufTNycNTN9p5aicfjv, Similarity Score: 0.7341395616531372
User ID: 6mDvGWpmuiCzNoWqkLfP, Similarity Score: 0.734026312828064
User ID: Gt6OGPcCTzzz2waskmn2, Similarity Score: 0.7308772802352905
User ID: bhKPo4Q17hA3pQlL4krW, Similarity Score: 0.7248979806900024
User ID: p63opsqfowndufII6PKc, Similarity Score: 0.7194515466690063
User ID: rBrN8xN4Efj2UizHcl3u, Similarity Score: 0.7171743512153625
User ID: 4KWqlqzr1ND8I8JFD6PZ, Similarity Score: 0.7124153971672058
User ID: QQo9A33OWZTDZrR4M7MA, Similarity Score: 0.707081139087677
User ID: KHk3qbuznly3kliNFreQ, Similarity Score: 0.